In [ ]:
"""
서브워드(subword) 분절 기법
    - 하나의 단어는 의미있는 여러 내부 단어(subword)의 조합으로 구성된 경우가 많음
    - ex) "confidence" > "con", "f", "er", "ence"
    - 단어 분리 작업은 토크나이징 전처리 과정에서 많이 활용함
    - 한국어, 일본어, 영어 등 언어에 구애받지 않고, 별도의 문법 없이 조사 구분이 가능함
    - 장점
        - OOV 문제에 대해 어느 정도 대처할 수 있도록 함
        - 단어가 많아질수록 학습해야 할 양이 늘어나는데, 이를 줄일 수 있음
"""

In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)


2.2.0


In [2]:
# Double check TF 2.0x is installed. If you ran the above block, there was a 
# 'reset all runtimes' button at the bottom that you needed to press
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
# If the import fails, run this
# !pip install -q tensorflow-datasets

import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)


In [4]:
train_data, test_data = imdb['train'], imdb['test']

In [5]:
tokenizer = info.features['text'].encoder
print(tokenizer.subwords)

, 'pee', 'nar', 'location_', 'ining_', 'gam', 'disappointing_', 'desire_', 'criminal_', 'considera', 'century_', 'celebrat', 'brow', 'area', 'Thin', 'Rec', "' (", 'ward_', 'vision_', 'treme', 'surprising_', 'super_', 'risk', 'receive', 'qual', 'pic', 'mee', 'levels', 'kins', 'jack', 'ire_', 'introduc', 'hits_', 'happening_', 'handsome', 'gradua', 'giv', 'garbage', 'forces_', 'finest_', 'easi', 'depressing', 'credits', 'asto', 'Sadly', 'Ple', 'Inc', 'Dick_', 'Alexand', 'wooden_', 'wood_', 'stro', 'steal_', 'soul_', 'reference', 'race', 'quis', 'pir', 'perv', 'obvious', 'majority_', 'lean', 'kes_', 'insti', 'identity', 'everybody_', 'double_', 'dies', 'credit', 'const', 'confe', 'compar', 'centur', 'bloody_', 'Under', 'Twi', 'Sean_', 'Lio', 'Halloween', 'Gal', 'Clu', 'Came', 'Barbara_', '?)', '11_', 'ws', 'ulous', 'subtle', 'substance', 'string', 'shocking_', 'scientist_', 'rian', 'nou', 'multi', 'lf', 'inal', 'harsh', 'handed', 'fir', 'expectations_', 'excited', 'exceptional', 'eva', 'c

In [6]:
sample_string = 'TensorFlow, from basics to mastery'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))


Tokenized string is [6307, 2327, 4043, 2120, 2, 48, 4249, 4429, 7, 2652, 8050]
The original string: TensorFlow, from basics to mastery


In [7]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
2 ----> , 
48 ----> from 
4249 ----> basi
4429 ----> cs 
7 ----> to 
2652 ----> master
8050 ----> y


In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_data.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_data.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_data))

In [9]:
embedding_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(), # Flatten()은 에러 발생(None 때문)
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 524,237
Trainable params: 524,237
Non-trainable params: 0
_________________________________________________________________


In [10]:
num_epochs = 10

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit(train_data, epochs=num_epochs, validation_data=test_data)

Epoch 1/10


ValueError: in user code:

    C:\Users\hykim\AppData\Local\Continuum\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\hykim\AppData\Local\Continuum\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\hykim\AppData\Local\Continuum\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\hykim\AppData\Local\Continuum\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\hykim\AppData\Local\Continuum\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:5

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, tokenizer.vocab_size):
  word = tokenizer.decode([word_num])
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')